<a href="https://colab.research.google.com/github/maximlambov/MARK/blob/master/TensorflowJS_Template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Run cell (commands) below and then hit F5 (refresh the page) right after that

In [ ]:
!npm install -g --unsafe-perm ijavascript
!ijsinstall --install=global
!jupyter-kernelspec list

In [ ]:
var { spawn } = require('child_process');
var sh = (cmd) => { 
    $$.async();
    var sp = spawn(cmd, { cwd: process.cwd(), stdio: 'pipe', shell: true, encoding: 'utf-8' });
    sp.stdout.on('data', data => console.log(data.toString()));
    sp.stderr.on('data', data => console.error(data.toString()));
    sp.on('close', () => $$.done());
};
var run_async = async (pf) => {
  $$.async();
  await pf();
  $$.done();
};
sh('npm init -y');

In [ ]:
sh('node -v; npm -v');

In [ ]:
sh('npm install @tensorflow/tfjs-node-gpu node-fetch@2');

In [ ]:
var tf = require('@tensorflow/tfjs-node-gpu');
tf.backend().isUsingGpuDevice;
tf.enableProdMode();

In [51]:
var normalization = ({ Xmin, Xmax, d1, d2 }) => {
  var maxToMin = (x) => {
    return ((x - Xmin) * (d2 - d1)) / (Xmax - Xmin) + d1;
  };

  var minToMax = (y) => {
    return ((y - d1) * (Xmax - Xmin)) / (d2 - d1) + Xmin;
  };

  var normalize = (data, excludeKeys) => {
    return data.map(item => {
      var preparedItem = Object.entries(item);

      return preparedItem.reduce((acc, item) => {
        if (excludeKeys?.includes(item[0])) {
          return {
            ...acc,
          };
        }
        return {
          ...acc,
          [item[0]]: maxToMin(+item[1]),
        };
      }, {});
    });
  };

  var denormalize = (data, excludeKeys) => {
    return data.map(item => {
      var preparedItem = Object.entries(item);

      return preparedItem.reduce((acc, item) => {
        if (excludeKeys?.includes(item[0])) {
          return acc;
        }
        return {
          ...acc,
          [item[0]]: minToMax(+item[1]),
        };
      }, {});
    });
  };

  return {
    normalize,
    denormalize,
    maxToMin,
    minToMax,
  };
};

var calculateBatchByPercentage = (len, percentage = 0.7, batchSize = 25) => {
  var batchLen = len / 25;
  return Math.floor(batchLen * percentage) * batchSize;
};

var arrayToTensor = (payload) =>
  tf.tensor(payload, [1, payload.length]);


In [ ]:
var fetch = require('node-fetch');

var EPOCHS = 3000;

$$.async();
(async () => {
  try {
    $$.async();
    var response = await fetch("https://query1.finance.yahoo.com/v8/finance/chart/AUDUSD=X?interval=1d&range=10y");

    var {chart} = await response.json();
    $$.done();

    let timestamp = chart.result[0].timestamp;

    // let adjustedClose = chart.result[0].indicators.adjclose[0].adjclose;
    // let adjustedClose = chart.result[0].indicators.quote[0].open;
    let adjustedClose = chart.result[0].indicators.quote[0].close;
    // let adjustedClose = chart.result[0].indicators.quote[0].low;
    // let adjustedClose = chart.result[0].indicators.quote[0].high;

    // --------------------------------
    const badIndexes = adjustedClose.reduce(
      (acc, item, index) => (typeof item !== 'number' ? [...acc, index] : acc),
      []
    );

    badIndexes.forEach(index => {
      timestamp.splice(index, 1);
      adjustedClose.splice(index, 1);
    });

    const isNormal = adjustedClose.length % 25 === 0;

    if (!isNormal) {
      adjustedClose = adjustedClose.splice(Math.ceil(adjustedClose.length % 25));
      timestamp = timestamp.splice(Math.ceil(timestamp.length % 25));
    }

    var { minToMax, maxToMin } = normalization({
      Xmin: Math.min(...adjustedClose.flat()),
      Xmax: Math.max(...adjustedClose.flat()),
      d1: 0,
      d2: 1,
    });


    var model = tf.sequential();

    model.add(
      tf.layers.dense({
        name: 'InputLayer',
        units: 300,
        inputShape: [20],
        activation: 'sigmoid',
      })
    );

    model.add(
      tf.layers.dense({
        name: 'HiddenLayer2',
        units: 300,
        activation: 'sigmoid',
      })
    );

    model.add(
      tf.layers.dense({ name: 'OutputLayer', units: 5, activation: 'sigmoid' })
    );

    model.summary();

    model.compile({
      loss: 'meanSquaredError',
      optimizer: tf.train.sgd(0.3),
    });

    var trainLength = calculateBatchByPercentage(adjustedClose.length);

    for (var i = 0; i < EPOCHS; i++) {
      for (var j = 0; j < trainLength; j += 5) {
        var trainX = adjustedClose.slice(j, j + 20);
        var trainY = adjustedClose.slice(j + 20, j + 25);

        var trainXBatch = trainX.map(i => +maxToMin(i).toFixed(4));
        var trainYBatch = trainY.map(i => +maxToMin(i).toFixed(4));

        $$.async()
        await model.fit(arrayToTensor(trainXBatch), arrayToTensor(trainYBatch), {
          epochs: 1,
        });
        $$.done()
      }
    }

    var csv = [];
    var dates = [];
    var real = [];
    var predict = [];
    var specials = [];
    var shift = 20; //  first 20 days are not predicted

    for (var j = trainLength; j < adjustedClose.length; j += 5) {
      var testX = adjustedClose.slice(j, j + 20);
      var testY = adjustedClose.slice(j + 20, j + 25);

      var testXBatch = testX.map(i => +maxToMin(i).toFixed(4));
      var testYBatch = testY.map(i => +maxToMin(i).toFixed(4));

      if (testYBatch.length !== 5) {
        break;
      }

      [...testY].forEach((_, index) => {
        var calculatedDate = timestamp[j + index + shift];

        dates.push(new Date(calculatedDate * 1000).toISOString().split('T')[0]);
      });

      var prediction = model.predict(arrayToTensor(testXBatch));
      predict.push(...prediction.dataSync().map(minToMax));

      real.push(...testYBatch.map(minToMax));

      if (j === adjustedClose.length - 25) {
        var data = [...testXBatch.slice(5, 20), ...testYBatch].map(
          item => +item.toFixed(4)
      );

      var prediction = model.predict(arrayToTensor(data));
      specials.push(...prediction.dataSync().map(minToMax));

      var data2 = [
        ...testXBatch.slice(10, 20),
        ...testYBatch,
        ...specials.map(maxToMin),
      ].map(item => +item.toFixed(4));

      var prediction2 = model.predict(arrayToTensor(data2));
      specials.push(...prediction2.dataSync().map(minToMax));
      }
    }
    console.log(`|   date   |     predict     |     real     |`);

    dates.forEach((date, index) => {
      // csv.push(`${date}, ${predict[index]}, ${real[index]}`);
      console.log(`${date}, ${predict[index]}, ${real[index]}`);
    });

    var lastDate = new Date(dates.at(-1));

    specials.forEach((special, i) => {
      var nextDate = new Date(lastDate.getTime() + (i + 1) * 24 * 60 * 60 * 1000)
      .toISOString()
      .split('T')[0];
      // csv.push(`${nextDate}, ${special}, undefined`);
        console.log(`${nextDate}, ${special}, undefined`);
    });


  } catch (e) {
    console.log("e",e)
   }

})()
$$.done();

